In [1]:
import librosa
from librosa import display
from PIL import Image
from matplotlib import pyplot
from numpy import asarray
import matplotlib.pyplot as plt
import numpy as np
import cmath
import seaborn as sns
import scipy
import IPython.display as ipd
import math
from numpy.linalg import inv

In [2]:
def NMFmodel(A,no_of_basis_vectors):
    np.random.seed(1)
    li=0
    Ws=np.random.rand(len(A),no_of_basis_vectors)
    Hs=np.random.rand(no_of_basis_vectors,len(A[0]))
#     Ws=np.ones((len(A),no_of_basis_vectors))*1000
#     Hs=np.ones((no_of_basis_vectors,len(A[0])))*50
    obj= 0.5* np.trace((A-Ws.dot(Hs)).T.dot(A-Ws.dot(Hs)))
    for i in range(100000):
        Hs=Hs * (np.dot(Ws.T,A)+li)/(Ws.T.dot(Ws).dot(Hs)+li)
        Ws=Ws*(np.dot(A,Hs.T)+li)/(Ws.dot(Hs).dot(Hs.T)+li)
        obj1=0.5* np.trace((A-Ws.dot(Hs)).T.dot(A-Ws.dot(Hs)))
        if np.allclose(obj,obj1, rtol=1e-7,atol=1e-7):
#             print(abs(obj-obj1),i)
            break
        else:
            obj=obj1
    return(Ws,Hs,i,abs(obj-obj1))

In [3]:
import scipy.io
eeg = scipy.io.loadmat('eeg.mat')

In [4]:
x_train=eeg['x_train'] 
xtest=eeg['x_te']
ytest=eeg['y_te']
ytrain=eeg['y_train']
ytest.shape
rank=[2,4,6,8,10,15,20]
k_near=[4,7,10,12,15,20,25]
flattened_y_train=list(ytrain.flatten())
flattened_y_test=list(ytest.flatten())

In [5]:
Tr_matrix=np.empty((255,0)) ### Training X_matrix

## Converting the training signal into STFT

In [6]:
for i in range(x_train.shape[2]):
    channel1=librosa.stft(x_train[:,0,i], n_fft = 64, hop_length=48, window='blackman')
    channel2=librosa.stft(x_train[:,1,i], n_fft = 64, hop_length=48, window='blackman')
    channel3=librosa.stft(x_train[:,2,i], n_fft = 64, hop_length=48, window='blackman')
    new_channel1=channel1[2:7,:].reshape(1,85)
    new_channel2=channel2[2:7,:].reshape(1,85)
    new_channel3=channel3[2:7,:].reshape(1,85)
    chanel=np.concatenate((new_channel1,new_channel2,new_channel3),axis=1)
    Tr_matrix=np.concatenate((Tr_matrix,chanel.T),axis=1)

In [7]:
Test_matrix=np.empty((255,0)) #### Test X matrix


## Converting the Test signal into STFT

In [8]:
for i in range(xtest.shape[2]):
    channel1=librosa.stft(x_train[:,0,i], n_fft = 64, hop_length=48, window='blackman')
    channel2=librosa.stft(x_train[:,1,i], n_fft = 64, hop_length=48, window='blackman')
    channel3=librosa.stft(x_train[:,2,i], n_fft = 64, hop_length=48, window='blackman')
    new_channel1=channel1[2:7,:].reshape(1,85)
    new_channel2=channel2[2:7,:].reshape(1,85)
    new_channel3=channel3[2:7,:].reshape(1,85)
    chanel=np.concatenate((new_channel1,new_channel2,new_channel3),axis=1)
    Test_matrix=np.concatenate((Test_matrix,chanel.T),axis=1)

In [9]:
print(Test_matrix)

[[-0.18705301+2.86229374e-17j -0.15800765+6.93889390e-18j
  -1.27693081-1.11022302e-16j ... -0.58106261+5.55111512e-17j
  -0.61230779+5.55111512e-17j  0.08088275+6.93889390e-18j]
 [-0.16769905-2.59186894e-01j  0.0889667 -1.24706089e-01j
  -0.14289826-3.93401951e-01j ... -0.62742352+3.13419968e-01j
   0.45533773-2.22693563e-01j -0.0134242 +4.36687857e-01j]
 [-0.10207901+5.35322487e-01j -0.16498095-1.48869693e-01j
   0.35892558-1.67546049e-01j ... -0.59812766+4.06445563e-02j
   0.461844  -1.75214589e-01j  0.27543956-7.70760551e-02j]
 ...
 [ 0.03431823+1.41867369e-01j -0.19715536+3.11366618e-01j
   0.1337263 -4.81889904e-01j ...  0.0291795 -3.16843502e-02j
  -0.02115282+2.86981426e-02j  0.06142748-2.42771596e-01j]
 [ 0.41499385-2.70438492e-01j -0.31009653+3.88155341e-01j
   0.35470533-3.85713816e-01j ...  0.48070675+2.94662505e-01j
   0.38798332-1.94062874e-01j -0.43758199-2.87101157e-02j]
 [ 0.61479318+3.91500533e-01j  0.21177115+1.96820498e-01j
  -0.64364648-3.26762050e-01j ...  0.18453

In [10]:
def sunny(Tr_matrix,Test_matrix,rank,k_nearest,flattened_y_test,flattened_y_train):
    W_tr,H_tr,k1,err1=NMFmodel(abs(Tr_matrix),rank)
    W_te,H_te,k2,err2=NMFmodel(abs(Test_matrix),rank)
    final_categories=[]
    accuracy_list=[]
    for i in range(len(H_te[0])):##28
        dis=[]
        for j in range(len(H_tr[0])):##112
            k=np.linalg.norm(H_tr[:,j]-H_te[:,i])
            dis.append(k)
        sorted_dist_comingfromwhichcolumn=list(np.argsort(dis)[0:k_nearest])
        listofcategoriesfora_i_point=[flattened_y_train[i] for i in sorted_dist_comingfromwhichcolumn]
        if (listofcategoriesfora_i_point.count(1)) > (listofcategoriesfora_i_point.count(2)):
            category=1
        else:
            category=2
        
        final_categories.append(category)
        
        if category==flattened_y_test[i]:
            flag=1
        else:
            flag=0
        accuracy_list.append(flag)
         
    final_accuracy=np.sum(accuracy_list)/len(accuracy_list)
    return(np.float(final_accuracy))

In [11]:
list_of_table=np.zeros((len(k_near),len(rank)))
for i in range(len(k_near)):
    for j in range(len(rank)):
        list_of_table[i][j]=sunny(Tr_matrix,Test_matrix,rank[j],k_near[i],flattened_y_test,flattened_y_train)
        print(list_of_table[i][j],i)
        

0.39285714285714285 0
0.5 0
0.6428571428571429 0
0.32142857142857145 0
0.39285714285714285 0
0.5714285714285714 0
0.42857142857142855 0
0.5 1
0.4642857142857143 1
0.6428571428571429 1
0.35714285714285715 1
0.42857142857142855 1
0.5357142857142857 1
0.39285714285714285 1
0.5357142857142857 2
0.4642857142857143 2
0.6071428571428571 2
0.42857142857142855 2
0.42857142857142855 2
0.5714285714285714 2
0.39285714285714285 2
0.5 3
0.5 3
0.5357142857142857 3
0.39285714285714285 3
0.4642857142857143 3
0.5357142857142857 3
0.4642857142857143 3
0.4642857142857143 4
0.5357142857142857 4
0.5714285714285714 4
0.32142857142857145 4
0.39285714285714285 4
0.6428571428571429 4
0.4642857142857143 4
0.35714285714285715 5
0.4642857142857143 5
0.6071428571428571 5
0.39285714285714285 5
0.39285714285714285 5
0.6071428571428571 5
0.39285714285714285 5
0.42857142857142855 6
0.4642857142857143 6
0.5357142857142857 6
0.39285714285714285 6
0.42857142857142855 6
0.5714285714285714 6
0.39285714285714285 6


In [12]:
print(list_of_table)

[[0.39285714 0.5        0.64285714 0.32142857 0.39285714 0.57142857
  0.42857143]
 [0.5        0.46428571 0.64285714 0.35714286 0.42857143 0.53571429
  0.39285714]
 [0.53571429 0.46428571 0.60714286 0.42857143 0.42857143 0.57142857
  0.39285714]
 [0.5        0.5        0.53571429 0.39285714 0.46428571 0.53571429
  0.46428571]
 [0.46428571 0.53571429 0.57142857 0.32142857 0.39285714 0.64285714
  0.46428571]
 [0.35714286 0.46428571 0.60714286 0.39285714 0.39285714 0.60714286
  0.39285714]
 [0.42857143 0.46428571 0.53571429 0.39285714 0.42857143 0.57142857
  0.39285714]]


The best accuracy obtained was 64.285% for different combinations of k and rank (combination1:rank=6 and k=20, combination2: rank=15 and k=15. The Ws and Hs for both train and test were initialized with random values.I have printed out the best accuracy result and as seen the highest accuracy is for rank=20 and k=20.

Below are the values values that I got for different k and ranks:

<table>
  <thead>
    <tr>
      <th>K</th>
      <th>Accuracies for different ranks(2,4,6,8,10,15,20). The values are in decimals and need to be converted into %  by multiplying with 100</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>4</td>
      <td>[0.39, 0.5, 0.64, 0.3214, 0.3928, 0.5714, 0.4285]</td>
    </tr>
    <tr>
      <td>7</td>
      <td>[[0.5, 0.4642, 0.6428, 0.3571, 0.42857143, 0.53571429, 0.39285714]</td>
    </tr>
    <tr>
      <td>10</td>
      <td>[0.53571429, 0.46428571, 0.60714286, 0.42857143, 0.42857143, 0.57142857, 0.39285714]</td>
    </tr>
    <tr>
      <td>12</td>
      <td>[0.5, 0.5, 0.53571429, 0.39285714, 0.46428571, 0.53571429, 0.46428571]</td>
    </tr>
    <tr>
      <td>15</td>
      <td>[0.46428571, 0.53571429, 0.57142857, 0.32142857, 0.39285714, 0.64285714, 0.46428571]</td>
    </tr>
    <tr>
      <td>20</td>
      <td>[[0.35714286, 0.46428571, 0.60714286, 0.39285714, 0.39285714, 0.60714286, 0.39285714]</td>
    </tr>
    <tr>
      <td>25</td>
      <td>[0.42857143, 0.46428571, 0.53571429, 0.39285714, 0.42857143, 0.57142857, 0.39285714]</td>
    </tr>
      
  </tbody>
</table>